In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.statistical import CopulaSimulation
from teslakit.mda import MaxDiss_Simplified_NoThreshold



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('KWAJALEIN')


In [3]:
# --------------------------------------
# load data and set parameters

_, xds_TCs_r2_params = db.Load_TCs_r2()  # TCs parameters inside radius 2

# TCs random generation and MDA parameters
num_sim_rnd = 100000
num_sel_mda = 1000



## Historical TCs - Probabilistic Simulation

In [4]:
# --------------------------------------
# Probabilistic simulation Historical TCs

# aux function
def FixPareto(var):
    'Fix data. It needs to start at 0 for Pareto adjustment '
    var = var.astype(float)
    mx = np.amax(var)
    aux = mx + np.absolute(var - mx)
    var_pareto = aux - np.amin(aux) + 0.00001
    
    return var_pareto, np.amin(aux)


# use small radius parameters (4º)
pmean = xds_TCs_r2_params.pressure_mean.values[:]
pmin = xds_TCs_r2_params.pressure_min.values[:]
gamma = xds_TCs_r2_params.gamma.values[:]
delta = xds_TCs_r2_params.delta.values[:]
vmean = xds_TCs_r2_params.velocity_mean.values[:]

# fix pressure for p
## Database and Site parametersareto adjustment (start at 0)
pmean_p, auxmin_pmean = FixPareto(pmean)
pmin_p, auxmin_pmin = FixPareto(pmin)

# join storm parameters for copula simulation
storm_params = np.column_stack(
    (pmean_p, pmin_p, gamma, delta, vmean)
)

# statistical simulate PCs using copulas 
kernels = ['GPareto', 'GPareto', 'ECDF', 'ECDF', 'ECDF']
storm_params_sim = CopulaSimulation(storm_params, kernels, num_sim_rnd)

# store simulated storms - parameters 
xds_TCs_r2_sim_params = xr.Dataset(
    {
        'pressure_mean':(('storm'), storm_params_sim[:,0] + auxmin_pmean),
        'pressure_min':(('storm'), storm_params_sim[:,1] + auxmin_pmin),
        'gamma':(('storm'), storm_params_sim[:,2]),
        'delta':(('storm'), storm_params_sim[:,3]),
        'velocity_mean':(('storm'), storm_params_sim[:,4]),
    },
    coords = {
        'storm':(('storm'), np.arange(num_sim_rnd))    
    },
)
print(xds_TCs_r2_sim_params)

db.Save_TCs_r2_sim_params(xds_TCs_r2_sim_params)


<xarray.Dataset>
Dimensions:        (storm: 100000)
Coordinates:
  * storm          (storm) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables:
    pressure_mean  (storm) float64 1.01e+03 1.022e+03 ... 1.037e+03 1.017e+03
    pressure_min   (storm) float64 1.01e+03 1.027e+03 ... 1.045e+03 1.018e+03
    gamma          (storm) float64 111.8 205.4 103.6 111.3 ... 136.1 0.0 120.9
    delta          (storm) float64 140.1 106.4 121.1 155.5 ... 207.7 88.01 152.5
    velocity_mean  (storm) float64 12.6 7.57 27.7 33.1 ... 14.28 20.3 14.28



## Historical TCs - MaxDiss classification

In [5]:
# --------------------------------------
# MaxDiss classification

## Historical TCs - Probabilistic Simulation
# get simulated parameters  
pmean_s = xds_TCs_r2_sim_params.pressure_mean.values[:]
pmin_s = xds_TCs_r2_sim_params.pressure_min.values[:]
gamma_s = xds_TCs_r2_sim_params.gamma.values[:]
delta_s = xds_TCs_r2_sim_params.delta.values[:]
vmean_s = xds_TCs_r2_sim_params.velocity_mean.values[:]

# subset, scalar and directional indexes
data_mda = np.column_stack((pmean_s, pmin_s, vmean_s, delta_s, gamma_s))
ix_scalar = [0,1,2]
ix_directional = [3,4]

centroids = MaxDiss_Simplified_NoThreshold(
    data_mda, num_sel_mda, ix_scalar, ix_directional 
)


# store MDA storms - parameters 
xds_TCs_r2_MDA_params = xr.Dataset(
    {
        'pressure_mean':(('storm'), centroids[:,0]),
        'pressure_min':(('storm'), centroids[:,1]),
        'gamma':(('storm'), centroids[:,2]),
        'delta':(('storm'), centroids[:,3]),
        'velocity_mean':(('storm'), centroids[:,4]),
    },
    coords = {
        'storm':(('storm'), np.arange(num_sel_mda))    
    },
)
print(xds_TCs_r2_MDA_params)

db.Save_TCs_r2_mda_params(xds_TCs_r2_MDA_params)



MaxDiss waves parameters: 100000 --> 1000

   MDA centroids: 1000/1000

<xarray.Dataset>
Dimensions:        (storm: 1000)
Coordinates:
  * storm          (storm) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
Data variables:
    pressure_mean  (storm) float64 1.138e+03 1.013e+03 ... 1.039e+03 1.031e+03
    pressure_min   (storm) float64 1.213e+03 1.014e+03 ... 1.038e+03 1.043e+03
    gamma          (storm) float64 20.71 8.028 34.86 6.622 ... 9.742 29.35 12.37
    delta          (storm) float64 139.4 309.8 127.2 128.2 ... 221.1 286.8 33.39
    velocity_mean  (storm) float64 97.78 207.9 0.0 111.1 ... 139.5 108.3 0.0
